# Benchmarking Our Algorithm against others

In [6]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']

        print('Building the environment...')
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [27]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 120

# Specify the Algorithms
algs = {  
        'SIPP':lambda HG: sequential_HybridSIPP(HG),
        "Seq_Tube": lambda HG: sequential_planning(Tube_Planning,HG.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [36]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [10,20,25]

In [37]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Room Alg: SIPP N agent: 10
0
1
2
3
4
5
6
7
8
MILP low-level encounters infeasibility.
9
On time for algorithm SIPP N agent =  10
Env: Room Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube N agent =  10
Building the environment...
Env: Room Alg: SIPP N agent: 20
0
1
2
3
4
5
6
7
8
MILP low-level encounters infeasibility.
9
10
11
12
13
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
14
MILP low-level encounters infeasibility.
15
16
17
18
19
On time for algorithm SIPP N agent =  20
Env: Room Alg: Seq_Tube N agent: 20
Planning for agent 0/20
Planning for agent 1/20
Planning for agent 2/20
Planning for agent 3/20
Planning for agent 4/20
Planning for agent 5/20
Planning for agent 6/2

In [21]:
env_name = 'Warehouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [9,12,15]

In [22]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Warehouse Alg: SIPP N agent: 9
0
1
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.


KeyboardInterrupt: 

In [29]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [30]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: MultiTunnel Alg: SIPP N agent: 3
0
1
MILP low-level encounters infeasibility.
2
MILP low-level encounters infeasibility.
On time for algorithm SIPP N agent =  3
Env: MultiTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Building the environment...
Env: MultiTunnel Alg: SIPP N agent: 6
0
1
MILP low-level encounters infeasibility.
2
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
3
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
4
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
5
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
On time for algorithm SIPP N agent =  6
Env: MultiTunnel Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for

In [25]:
env_name = 'SingleTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [26]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: SingleTunnel Alg: SIPP N agent: 3
0
1
MILP low-level encounters infeasibility.
2
MILP low-level encounters infeasibility.
On time for algorithm SIPP N agent =  3
Env: SingleTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Building the environment...
Env: SingleTunnel Alg: SIPP N agent: 6
0
1
2
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
3
MILP low-level encounters infeasibility.
4
MILP low-level encounters infeasibility.
5
MILP low-level encounters infeasibility.
On time for algorithm SIPP N agent =  6
Env: SingleTunnel Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube N agent =  6
Building the environment...
Env: SingleTunnel Alg: SIPP N agent: 9
0
1
2
MILP low-level encounters i